In [4]:
import pandas as pd

In [5]:
f = pd.read_csv('../Raw_Datasets/items_Giacomo/items.csv',sep=',')
f['TIPO'].unique()

array(['FUENTE', 'INSTALACIÓN DEPORTIVA', 'PISCINA', 'ZONA VERDE',
       'SENDA', 'CENTRO DE SALUD', 'CICLOCARRIL', 'PASO DE BICI',
       'CARRIL BICI', 'VÍA MIXTA', 'SENDA CICLABLE', 'PISTA BICI',
       'ARCEN BICI', 'ACERA BICI', 'ENCAMINAMIENTO', 'CIRCUITO DE OCIO',
       'BANDA RECOMENDADA', 'GIRO PERMITIDO', 'SENTIDO PERMITIDO'],
      dtype=object)

In [6]:
import json
import csv

# Función para leer datos meteorológicos desde un archivo JSON
def leer_datos_meteorologicos(nombre_archivo):
    with open(nombre_archivo, 'r') as archivo:
        datos = json.load(archivo)
    return datos

# Función para etiquetar la presencia de una ola de calor basada en datos climáticos
def etiquetar_ola_de_calor(datos_clima, umbral_temperatura):
    etiquetas = []
    for registro in datos_clima['data']['list']:
        temperatura = registro['main']['temp']
        if temperatura > umbral_temperatura:
            etiquetas.append(1)  # Hay ola de calor
        else:
            etiquetas.append(0)  # No hay ola de calor
    return etiquetas

# Función para cargar datos de instalaciones desde un archivo CSV y filtrar por tipo de instalación
def cargar_datos_instalaciones(nombre_archivo_csv, tipo_instalacion):
    csv.field_size_limit(500 * 1024)
    datos_instalaciones = []
    with open(nombre_archivo_csv, 'r', encoding='utf-8',newline='') as archivo_csv:
        lector_csv = csv.DictReader(archivo_csv)
        for fila in lector_csv:
            if fila['TIPO'] == tipo_instalacion:
                datos_instalaciones.append(fila)
    return datos_instalaciones

def obtener_recomendaciones(datos_clima, etiquetas_ola_calor, datos_instalaciones, tiene_problemas_salud):
    # Implementa la lógica para generar las recomendaciones de deportes
    # basadas en las condiciones climáticas, la ubicación del usuario y los datos
    # de instalaciones deportivas disponibles.

    recomendaciones = []

    # Implementa la lógica para recomendar deportes seguros según si hay ola de calor o no.
    if etiquetas_ola_calor[0] == 1:  # Primer registro en los datos de etiquetas corresponde a la predicción actual
        # Recomendar actividades en centros de salud si tiene problemas de salud
        if tiene_problemas_salud:
            recomendaciones.extend(obtener_recomendaciones_centros_salud(datos_instalaciones))
    else:
        # Recomendar actividades al aire libre en parques y jardines
        recomendaciones.extend(obtener_recomendaciones_parques(datos_instalaciones))

    return recomendaciones

# Función para obtener recomendaciones específicas para parques y jardines
def obtener_recomendaciones_parques(instalaciones):
    recomendaciones_parques = []
    for instalacion in instalaciones:
        if instalacion['TIPO'] == 'parque':
            recomendaciones_parques.append(f"Hacer ejercicio en el parque: {instalacion['NOMBRE']} ({instalacion['DIRECCION']})")
    return recomendaciones_parques

# Función para obtener recomendaciones específicas para centros de salud
def obtener_recomendaciones_centros_salud(instalaciones):
    recomendaciones_centros_salud = []
    for instalacion in instalaciones:
        if instalacion['TIPO'].lower() == 'centro_salud' and tiene_problemas_salud == True:
            recomendaciones_centros_salud.append(f"Realizar actividad en centro de salud: {instalacion['NOMBRE']} ({instalacion['DIRECCION']})")
    return recomendaciones_centros_salud

if __name__ == "__main__":
    nombre_archivo_json = "../Raw_Datasets/Trabajado/tiempo.json"  # Reemplaza con el nombre de tu archivo JSON
    nombre_archivo_sitios = "../Raw_Datasets/items_Giacomo/items.csv"  # Reemplaza con el nombre de tu archivo CSV de sitios (piscinas, parques y polideportivos)

    umbral_temperatura_ola_calor = 30.0  # Reemplaza con el umbral de temperatura para definir una ola de calor
    tiene_problemas_salud = True  # Reemplaza con True si el usuario tiene problemas de salud, False si no

    datos_clima = leer_datos_meteorologicos(nombre_archivo_json)
    etiquetas_ola_calor = etiquetar_ola_de_calor(datos_clima, umbral_temperatura_ola_calor)

    tipos_instalaciones_deportivas = {'PISCINA', 'ZONA VERDE', 'SENDA', 'CENTRO DE SALUD'}

    datos_instalaciones = []
    for tipo in tipos_instalaciones_deportivas:
        datos_instalaciones.extend(cargar_datos_instalaciones(nombre_archivo_sitios, tipo))

    recomendaciones_deportes = obtener_recomendaciones(
        datos_clima, etiquetas_ola_calor, datos_instalaciones, tiene_problemas_salud
    )

    # Imprime las recomendaciones
    print("Recomendaciones de deportes:")
    for deporte in recomendaciones_deportes:
        print(deporte)


Recomendaciones de deportes:


In [26]:
import json
import csv
import pandas as pd
from sklearn.ensemble import RandomForestClassifier

# Función para leer datos meteorológicos desde un archivo JSON
def leer_datos_meteorologicos(nombre_archivo):
    with open(nombre_archivo, 'r') as archivo:
        datos = json.load(archivo)
    return datos

# Función para etiquetar la presencia de una ola de calor basada en datos climáticos
def etiquetar_ola_de_calor(datos_clima, umbral_temperatura):
    etiquetas = []
    for registro in datos_clima['data']['list']:
        temperatura = registro['main']['temp']
        if temperatura > umbral_temperatura:
            etiquetas.append(1)  # Hay ola de calor
        else:
            etiquetas.append(0)  # No hay ola de calor
    return etiquetas

# Función para cargar datos de instalaciones desde un archivo CSV y filtrar por tipo de instalación
def cargar_datos_instalaciones(nombre_archivo_csv, tipo_instalacion):
    csv.field_size_limit(500 * 1024)
    datos_instalaciones = []
    with open(nombre_archivo_csv, 'r', encoding='utf-8', newline='') as archivo_csv:
        lector_csv = csv.DictReader(archivo_csv)
        for fila in lector_csv:
            if fila['TIPO'] == tipo_instalacion:
                datos_instalaciones.append(fila)
    return datos_instalaciones

# Función para obtener recomendaciones de deportes utilizando el modelo de machine learning
def obtener_recomendaciones_deportes_modelo(datos_clima, etiquetas_ola_calor, datos_instalaciones):
    # Obtener los datos históricos de recomendaciones de deportes
    # Aquí asumimos que tienes un archivo CSV llamado 'datos_historicos_deportes.csv'
    # que contiene datos históricos con columnas como 'Temperatura', 'OlaDeCalor', 'TipoInstalacion' y 'Recomendacion'
    df_historico = pd.read_csv('../Raw_Datasets/datos_historicos_deportes.csv')

    # Características (features) del modelo: Temperatura y Ola de calor (1 si hay ola de calor, 0 si no)
    X = df_historico[['', '']].values

    # Etiquetas del modelo: Recomendación de deporte
    y = df_historico['Recomendacion'].values

    # Entrenar el modelo Random Forest
    modelo = RandomForestClassifier(random_state=42)
    modelo.fit(X, y)

    # Obtener los valores actuales de Temperatura y Ola de calor
    temperatura_actual = datos_clima['data']['list'][0]['main']['temp']
    ola_de_calor_actual = etiquetas_ola_calor[0]

    # Realizar la predicción del modelo con los datos actuales
    prediccion_actual = modelo.predict([[temperatura_actual, ola_de_calor_actual]])

    recomendaciones = []

    if prediccion_actual[0] == 1:
        # Si el modelo predice que hay que evitar deportes, entonces:
        # Recomendar actividades en centros de salud si tiene problemas de salud
        if tiene_problemas_salud:
            recomendaciones.extend(obtener_recomendaciones_centros_salud(datos_instalaciones))
    else:
        # Si el modelo predice que se pueden hacer deportes, entonces:
        # Recomendar actividades al aire libre en parques y jardines
        recomendaciones.extend(obtener_recomendaciones_parques(datos_instalaciones))

    return recomendaciones

# Función para obtener recomendaciones específicas para parques y jardines
def obtener_recomendaciones_parques(instalaciones):
    recomendaciones_parques = []
    for instalacion in instalaciones:
        if instalacion['TIPO'] == 'parque':
            recomendaciones_parques.append(f"Hacer ejercicio en el parque: {instalacion['NOMBRE']} ({instalacion['DIRECCION']})")
    return recomendaciones_parques

# Función para obtener recomendaciones específicas para centros de salud
def obtener_recomendaciones_centros_salud(instalaciones):
    recomendaciones_centros_salud = []
    for instalacion in instalaciones:
        if instalacion['TIPO'].lower() == 'centro_salud' and tiene_problemas_salud:
            recomendaciones_centros_salud.append(f"Realizar actividad en centro de salud: {instalacion['NOMBRE']} ({instalacion['DIRECCION']})")
    return recomendaciones_centros_salud

if __name__ == "__main__":
    nombre_archivo_json = "../Raw_Datasets/Trabajado/tiempo.json"  # Reemplaza con el nombre de tu archivo JSON
    nombre_archivo_sitios = "../Raw_Datasets/items_Giacomo/items.csv"  # Reemplaza con el nombre de tu archivo CSV de sitios (piscinas, parques y polideportivos)

    umbral_temperatura_ola_calor = 30.0  # Reemplaza con el umbral de temperatura para definir una ola de calor
    tiene_problemas_salud = True  # Reemplaza con True si el usuario tiene problemas de salud, False si no

    datos_clima = leer_datos_meteorologicos(nombre_archivo_json)
    etiquetas_ola_calor = etiquetar_ola_de_calor(datos_clima, umbral_temperatura_ola_calor)

    tipos_instalaciones_deportivas = {'PISCINA', 'ZONA VERDE', 'SENDA', 'CENTRO DE SALUD'}

    datos_instalaciones = []
    for tipo in tipos_instalaciones_deportivas:
        datos_instalaciones.extend(cargar_datos_instalaciones(nombre_archivo_sitios, tipo))

    # Obtener recomendaciones de deportes utilizando el modelo de machine learning
    recomendaciones_deportes_modelo = obtener_recomendaciones_deportes_modelo(
        datos_clima, etiquetas_ola_calor, datos_instalaciones
    )

    # Imprime las recomendaciones del modelo
    print("Recomendaciones de deportes (modelo):")
    for deporte in recomendaciones_deportes_modelo:
        print(deporte)


KeyError: "None of [Index(['Temperatura', 'OlaDeCalor'], dtype='object')] are in the [columns]"

In [11]:
X = pd.read_csv('../Raw_Datasets/items_Giacomo/deportes_final.csv',sep=',')
X['Actividad'].unique()

array(['BMX', 'Ciclismo', 'Ciclismo estacionario', 'Calistenia',
       'Entrenamiento en circuito', 'Levantamiento de peso',
       'Máquina escaladora', 'Aeróbicos', 'Yoga', 'Aeróbicos acuáticos',
       'Correr', 'Atletismo', 'Bádminton', 'Baloncesto', 'Bolos',
       'Billar', 'Boxeo', 'Dardos', 'Frisbee', 'Ultimate frisbee', 'Golf',
       'Minigolf', 'Gimnasia', 'Balonmano', 'Hockey', 'Petanca', 'Calva',
       'Artes marciales', 'Kickboxing', 'Kickball', 'Pádel', 'Raquetbol',
       'Escalada', 'Salto a la comba', 'Chito', 'Skateboard', 'Patinaje',
       'Fútbol', 'Fútbol sala', 'Sóftbol', 'Béisbol',
       'Tenis de mesa / ping pong', 'Tai chi', 'Tenis', 'Tenis en pareja',
       'Voleibol', 'Marcha rápida', 'Montañismo', 'Caminar', 'Natación',
       'Nado sincronizado', 'Waterpolo', 'Voleibol acuático',
       'Patinaje sobre hielo', 'Frontenis', 'Senderismo'], dtype=object)

In [38]:
import pandas as pd
import numpy as np
import random
import string
from datetime import datetime, timedelta

# Lista de deportes
deportes = np.array(['BMX', 'Ciclismo', 'Ciclismo estacionario', 'Calistenia', 'Entrenamiento en circuito',
                     'Levantamiento de peso', 'Máquina escaladora', 'Aeróbicos', 'Yoga', 'Aeróbicos acuáticos',
                     'Correr', 'Atletismo', 'Bádminton', 'Baloncesto', 'Bolos', 'Billar', 'Boxeo', 'Dardos',
                     'Frisbee', 'Ultimate frisbee', 'Golf', 'Minigolf', 'Gimnasia', 'Balonmano', 'Hockey',
                     'Petanca', 'Calva', 'Artes marciales', 'Kickboxing', 'Kickball', 'Pádel', 'Raquetbol',
                     'Escalada', 'Salto a la comba', 'Chito', 'Skateboard', 'Patinaje', 'Fútbol', 'Fútbol sala',
                     'Sóftbol', 'Béisbol', 'Tenis de mesa / ping pong', 'Tai chi', 'Tenis', 'Tenis en pareja',
                     'Voleibol', 'Marcha rápida', 'Montañismo', 'Caminar', 'Natación', 'Nado sincronizado',
                     'Waterpolo', 'Voleibol acuático', 'Patinaje sobre hielo', 'Frontenis', 'Senderismo'])

# Generar un DataFrame sintético con datos históricos de recomendaciones de deportes
np.random.seed(42)
num_datos = 1000

# Simular datos de Fecha y Hora
fechas = [datetime.now() - timedelta(days=i) for i in range(num_datos)]
horas = [random.randint(0, 23) for _ in range(num_datos)]
fechas_y_horas = [fecha.replace(hour=hora, minute=0, second=0) for fecha, hora in zip(fechas, horas)]

# Simular datos de Clima
clima = np.random.choice(['nublado', 'soleado', 'Lluvioso'], num_datos)

# Simular datos de Temperatura en el rango de 10 a 35 grados Celsius
temperatura = np.random.uniform(10, 35, num_datos)

# Simular datos de Humedad en el rango de 30 a 80 por ciento
humedad = np.random.randint(30, 81, num_datos)


# Simular datos de Edad en el rango de 18 a 65 años
edad = np.random.randint(18, 66, num_datos)

# Simular deportes que ha hecho antes
deporte_practicado = np.random.choice(deportes, num_datos)

# Simular datos de distancia en el rango de 0.5 a 20 kilómetros
distancia = np.random.uniform(0.5, 20, num_datos)

# Simular datos de peso en el rango de 50 a 100 kilogramos
peso = np.random.uniform(50, 100, num_datos)

# Simular objetivos de entrenamiento
objetivos_entrenamiento = np.random.choice(['Fuerza', 'Resistencia', 'Flexibilidad', 'Velocidad', 'Equilibrio'], num_datos)

# Simular condición física
condicion_fisica = np.random.choice(['Baja', 'Media', 'Alta'], num_datos)

calorias_gastadas = (distancia * 0.5 * peso).round(2)

# Generar nombres aleatorios para cada registro
def generar_nombre_aleatorio():
    return ''.join(random.choice(string.ascii_uppercase + string.ascii_lowercase) for _ in range(6))

nombres_aleatorios = [generar_nombre_aleatorio() for _ in range(num_datos)]

# Generar nombres de personas para cada registro
nombres_personas = np.random.choice(['John', 'Emily', 'Michael', 'Jessica', 'William', 'Sophia', 'Christopher', 'Emma', 'Daniel', 'Olivia'], num_datos)

# Crear el DataFrame
df_historico = pd.DataFrame({
    'Fecha': fechas_y_horas,
    'Nombres': nombres_personas,
    'Hora': horas,
    'Clima': clima,
    'Temperatura (°C)': temperatura.round().astype(int),
    'Humedad': humedad,
    'Edad': edad,
    'Deporte_practicado': deportes_anteriores,
    'Distancia (Km)': distancia.round(2),
    'Peso (Kg)': peso.round(2),
    'ObjetivoEntrenamiento': objetivos_entrenamiento,
    'CondicionFisica': condicion_fisica,
    'CaloriasGastadas': calorias_gastadas,
})

# Eliminar las columnas RecomendacionDeporte y RecomendacionTipoInstalacion
# df_historico.drop(columns=['RecomendacionDeporte', 'RecomendacionTipoInstalacion'], inplace=True)

# Guardar el DataFrame en un archivo CSV
df_historico.to_csv('../Raw_Datasets/datos_historicos_deportes.csv', index=False)


In [40]:
df_historico

,Fecha,Nombres,Hora,Clima,Temperatura (°C),Humedad,Edad,Deporte_practicado,Distancia (Km),Peso (Kg),ObjetivoEntrenamiento,CondicionFisica,CaloriasGastadas
0,2023-07-29 02:00:00.354617,Christopher,2,Lluvioso,12,59,58,Frontenis,3.90,94.30,Flexibilidad,Alta,183.77
1,2023-07-28 15:00:00.354617,Michael,15,Bueno,23,43,56,Pádel,17.57,90.80,Fuerza,Media,797.73
2,2023-07-27 21:00:00.354617,Daniel,21,Lluvioso,20,35,30,Tenis de mesa / ping pong,9.11,50.95,Equilibrio,Alta,232.17
3,2023-07-26 21:00:00.354617,Emma,21,Lluvioso,35,32,28,Aeróbicos acuáticos,19.69,86.80,Equilibrio,Alta,854.62
4,2023-07-25 00:00:00.354617,Michael,0,Bueno,13,45,63,Raquetbol,8.22,60.42,Resistencia,Alta,248.21
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,2020-11-06 09:00:00.355620,Christopher,9,Malo,33,46,53,Caminar,6.99,69.33,Equilibrio,Baja,242.30
996,2020-11-05 03:00:00.355620,Daniel,3,Malo,13,53,49,Boxeo,11.37,91.06,Velocidad,Alta,517.63
997,2020-11-04 18:00:00.355620,Christopher,18,Lluvioso,30,50,49,Tenis de mesa / ping pong,14.29,98.47,Flexibilidad,Alta,703.44
998,2020-11-03 22:00:00.355620,John,22,Lluvioso,20,80,21,Kickball,13.42,72.70,Equilibrio,Baja,487.79


In [24]:
import os

def get_file_type(file_path):
    # Obtenemos la extensión del archivo
    file_extension = os.path.splitext(file_path)[1].lower()
    
    # Diccionario de extensiones y sus tipos correspondientes
    file_types = {
        '.csv': 'CSV',
        '.xlsx': 'Excel',
        '.xls': 'Excel',
        '.sas7bdat': 'SAS',
        '.sav': 'SPSS',
        '.dta': 'Stata',
        '.json': 'JSON',
        '.xml': 'XML',
        '.txt': 'Texto',
        '.sql': 'SQL',
        # Agrega aquí más extensiones y sus tipos correspondientes si es necesario
    }
    
    # Buscamos el tipo de archivo en el diccionario
    file_type = file_types.get(file_extension, 'Desconocido')
    
    return file_type

# Ejemplo de uso
file_path = '../Raw_Datasets/MD2833/ES2833'
tipo_archivo = get_file_type(file_path)
print(f"El archivo en '{file_path}' es de tipo: {tipo_archivo}")


El archivo en '../Raw_Datasets/MD2833/ES2833' es de tipo: Desconocido


In [25]:
def examinar_archivo(path):
    try:
        with open(path, 'r') as archivo:
            contenido = archivo.read()
            # Aquí puedes imprimir o analizar el contenido del archivo
            print(contenido)
    except Exception as e:
        print(f"Error al abrir el archivo: {e}")

# Reemplaza 'ruta_del_archivo' con la ubicación del archivo que deseas examinar
ruta_del_archivo = '../Raw_Datasets/MD2833/ES2833'
examinar_archivo(ruta_del_archivo)


DATA LIST FILE='DA2833'
      / CUES 5-9 A1 1-9(A) CCAA PROV 10-13 MUN 14-16 TAMUNI 17 AREA 18
     DISTR 19-20 SECCION 21-23 ENTREV 24-27
     P1 P2 28-29 P301 P302 30-33 P4 34 P5 35-36 P6 P7 37-38
     P801 P802 39-42 P901 P902 P1001 TO P1003 43-52 P1101 TO P1108
     P12 53-61 P12A01 P12A02 62-65 P13 TO P16 66-69 P1701 TO P1748
     P17A01 TO P17A47 P17B01 TO P17B47 70-211 ENTRE17 212 P18 P19
     P19A P20 TO P22 P22A P23 213-220 P24 P2501 TO P2504
     P2601 TO P2606 P26A01 TO P26A06 P27 P2801 TO P2804 221-242
     P29 P3001 TO P3006 P31 P31A P32 P32A01 TO P32A08 P33 243-261
     P33A01 P33A02 P33B01 P33B02 262-269 P3401 TO P3404 P35 270-274
     P3601 TO P3608 P36A01 TO P36A08 P36B01 TO P36B08 P37 P3801 TO P3805
     P39 P39A01 TO P39A08 P4001 P4002 P41 P41A01 TO P41A08 275-324
     P42 P43 P4401 TO P4403 P45 P4601 P4602 P47 P47A P48 P48A P49
     P50 TO P52 P52A  325-341
     P5301 P5302 342-343 P5401 TO P5403 344-349
     P55 P55A P56 P5701 P5702 P58 P5901 P5902 P60 P60A 350-359

In [ ]:
def examinar_archivo(path):
    try:
        with open(path, 'r') as archivo:
            contenido = archivo.read()
            # Aquí puedes imprimir o analizar el contenido del archivo
            print(contenido)
    except Exception as e:
        print(f"Error al abrir el archivo: {e}")

# Reemplaza 'ruta_del_archivo' con la ubicación del archivo que deseas examinar
ruta_del_archivo = '../Raw_Datasets/MD2833/'
examinar_archivo(ruta_del_archivo)